In [24]:
import pandas as pd
import statsapi
from pprint import pprint

In [2]:
 url = "http://www.espn.com/mlb/war/leaders/_/type/seasonal/year/20"

In [3]:
year_list = ['19', '18', '17', '16', '15']
df_list = []

In [4]:
for year in year_list:
    query_url = url+year
    qdfl = pd.read_html(query_url)
    qdfl[0]['season'] = year
    df_list.append(qdfl[0])

In [5]:
data = pd.concat(df_list)
data.head()

,0,1,2,3,4,5,6,7,8,9,season
0,2019 Season WAR Leaders,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19
1,RK,PLAYER,WAR,OFF,DEF,WAA,TRPG,ORPG,RAA,WAAWP,19
2,1,Cody Bellinger,9,6.6,1.9,7,5,4.7,72.9,0.5,19
3,2,Alex Bregman,8.4,7.7,1.2,6.1,5.3,5,62,0.5,19
4,3,Mike Trout,8.3,8.3,0.3,6.3,5.5,5,65.7,0.5,19


In [6]:
cols = ['rank', 'player', 'WAR', 'OFF', 'DEF', 'WAA', 'TRPG', 'ORPG', 'RAA', 'WAAWP', 'Season']

In [7]:
data.columns = cols
del data['rank']
data.head()

,player,WAR,OFF,DEF,WAA,TRPG,ORPG,RAA,WAAWP,Season
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19
1,PLAYER,WAR,OFF,DEF,WAA,TRPG,ORPG,RAA,WAAWP,19
2,Cody Bellinger,9,6.6,1.9,7,5,4.7,72.9,0.5,19
3,Alex Bregman,8.4,7.7,1.2,6.1,5.3,5,62,0.5,19
4,Mike Trout,8.3,8.3,0.3,6.3,5.5,5,65.7,0.5,19


In [8]:
data = data.dropna(subset=['player'])

In [9]:
data = data[~data.player.str.contains("WAR")]

In [10]:
data.head()

,player,WAR,OFF,DEF,WAA,TRPG,ORPG,RAA,WAAWP,Season
1,PLAYER,WAR,OFF,DEF,WAA,TRPG,ORPG,RAA,WAAWP,19
2,Cody Bellinger,9,6.6,1.9,7,5,4.7,72.9,0.5,19
3,Alex Bregman,8.4,7.7,1.2,6.1,5.3,5,62,0.5,19
4,Mike Trout,8.3,8.3,0.3,6.3,5.5,5,65.7,0.5,19
5,Marcus Semien,8.1,7.5,1.5,5.8,5.3,5,58.5,0.5,19


In [39]:
# new data frame with split value columns 
new = data["player"].str.split(" ", n = 1, expand = True) 
  
# making separate first name column from new data frame 
data["First Name"]= new[0] 
data["Last Name"]= new[1] 
  
data = data.drop(data.index[0])
data.head()

,player,WAR,OFF,DEF,WAA,TRPG,ORPG,RAA,WAAWP,Season,First Name,Last Name
4,Mike Trout,8.3,8.3,0.3,6.3,5.5,5,65.7,0.5,19,Mike,Trout
5,Marcus Semien,8.1,7.5,1.5,5.8,5.3,5,58.5,0.5,19,Marcus,Semien
6,Justin Verlander,7.8,0,0,5.8,5,3.4,56.6,0.7,19,Justin,Verlander
7,Mike Minor,7.8,0,0,5.9,5,3.2,57,0.7,19,Mike,Minor
8,Lance Lynn,7.6,0,0,5.7,5,3.3,55.6,0.7,19,Lance,Lynn


In [40]:
players = [statsapi.lookup_player(x) for x in data['player']]
players[0]

[{'id': 545361,
  'fullName': 'Mike Trout',
  'firstName': 'Michael',
  'lastName': 'Trout',
  'primaryNumber': '27',
  'currentTeam': {'id': 108},
  'primaryPosition': {'code': '8', 'abbreviation': 'CF'},
  'useName': 'Mike',
  'boxscoreName': 'Trout',
  'nickName': 'Kiiiiid',
  'mlbDebutDate': '2011-07-08',
  'nameFirstLast': 'Mike Trout',
  'firstLastName': 'Mike Trout',
  'lastFirstName': 'Trout, Mike',
  'lastInitName': 'Trout, M',
  'initLastName': 'M Trout',
  'fullFMLName': 'Michael Nelson Trout',
  'fullLFMName': 'Trout, Michael Nelson'}]

## Everything below here is not working yet:

#### Trying to loop through players list to pull id's but getting an IndexError: list index out of range

In [20]:
players1 = [players[0][i] for i in range(len(players))]

IndexError: list index out of range

In [60]:
# for i in players:
#     pprint(i[0]['id'])

#### testing length/range of player list

In [55]:
# statsapi.lookup_player('Adrian Beltre')

[]

In [61]:
# pd.set_option('display.max_rows', 500) 
# data

In [49]:
range(len(players))

range(0, 500)

#### list comprehension to pull id's into list

In [ ]:
player_ids = [players[i]['id'] for range(len(players))]

#### adding id's to DF

In [ ]:
data['id'] = player_ids

#### creating empty list for records to be added into mongo

In [64]:
mongo_list = []

#### loop to get stat data

In [65]:
# for i in data.index:

    #creating url to scrape from ESPN
#query_url = 'http://www.espn.com/mlb/player/stats/_/id/' + data['id'][i] + '/' + data['First Name'][i] + '-' + data['Last Name'][i]
query_url = 'http://www.espn.com/mlb/player/stats/_/id/30836/mike-trout'
#add results to temporary df
temp_df = pd.read_html(query_url)
#above returns a list of dfs, this line pulls the df that contains the stat table
stat_df = temp_df[1]
#dropping the blank row at the top and the summary row at the bottom
stat_df.drop(stat_df.tail(1).index,inplace=True)
stat_df.drop(stat_df.head(1).index,inplace=True)
#turning the df into a dictionary and adding to the mongo listS
mongo_list.append(stat_df.to_dict('series'))

In [66]:
mongo_list

[{0: 1      YEAR
  2      2011
  3      2012
  4      2013
  5      2014
  6      2015
  7      2016
  8      2017
  9      2018
  10     2019
  11    Total
  Name: 0, dtype: object, 1: 1      TEAM
  2       LAA
  3       LAA
  4       LAA
  5       LAA
  6       LAA
  7       LAA
  8       LAA
  9       LAA
  10      LAA
  11    Total
  Name: 1, dtype: object, 2: 1       GP
  2       40
  3      139
  4      157
  5      157
  6      159
  7      159
  8      114
  9      140
  10     134
  11    1199
  Name: 2, dtype: object, 3: 1       AB
  2      123
  3      559
  4      589
  5      602
  6      575
  7      549
  8      402
  9      471
  10     470
  11    4340
  Name: 3, dtype: object, 4: 1       R
  2      20
  3     129
  4     109
  5     115
  6     104
  7     123
  8      92
  9     101
  10    110
  11    903
  Name: 4, dtype: object, 5: 1        H
  2       27
  3      182
  4      190
  5      173
  6      172
  7      173
  8      123
  9      147
  10     137
  11  